In [3]:
import os

In [4]:
%pwd

'c:\\Users\\aarav\\Downloads\\VisualSearch\\srcx\\research'

In [5]:
os.chdir('../')

In [18]:
%pwd

'c:\\Users\\aarav\\Downloads\\VisualSearch\\srcx'

In [19]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [20]:
from VisualSearch.constants import *
from VisualSearch.utils.common import read_yaml, create_directories

In [21]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):
        
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)
            self.schema = read_yaml(schema_filepath)

            create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
          config = self.config.data_ingestion 

          create_directories([config.root_dir])

          data_ingestion_config = DataIngestionConfig(
                root_dir=config.root_dir,
                source_URL=config.source_URL,
                local_data_file=config.local_data_file,
                unzip_dir=config.unzip_dir
          )       

          return data_ingestion_config

In [22]:
import os 
import urllib.request as request 
import zipfile
from VisualSearch import logger 
from VisualSearch.utils.common import get_size

In [23]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config   

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename= self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [27]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e   

[2024-02-01 15:52:26,807: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-01 15:52:26,810: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-01 15:52:26,812: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-02-01 15:52:26,813: INFO: common: created directory at: artifacts]
[2024-02-01 15:52:26,814: INFO: common: created directory at: artifacts/data_ingestion]
[2024-02-01 15:54:29,537: INFO: 4257903981: artifacts/data_ingestion/data.zip download! with following info: 
x-amz-id-2: 1uUNqiAJVe4/pT3zfb2HmgPtYjsxnnv6WytJGl9vxEFuxYEMREvZQMOQpVW/zwtLjHYgW75f1bB/gLg5KQgSPpbp5ykLpok2SDn3z/ZLAhc=
x-amz-request-id: J1NSV71D38NGVQCP
Date: Thu, 01 Feb 2024 10:22:28 GMT
Last-Modified: Thu, 01 Feb 2024 09:45:15 GMT
ETag: "4a8a39b999b7d9017f796149c164d6a0-18"
x-amz-server-side-encryption: AES256
Accept-Ranges: bytes
Content-Type: application/zip
Server: AmazonS3
Content-Length: 292922571
Connection: close

]
